In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('../small_business/data/restaurants.csv')
data = data.drop(columns = 'Unnamed: 0')

In [6]:
data['type'].unique()

array(['brunch', 'cafe', 'mediterranean', 'european', 'chicken', 'bar',
       'seafood', 'south_america', 'veggie_healthy', 'bistro', 'grill',
       'indian_nepalese', 'portuguese', 'pizza', 'japanese', 'pasta',
       'mexican', 'africa_me', 'burger', 'italian', 'asian', 'pastry',
       'fast_food', 'fado'], dtype=object)

In [ ]:
maybe: brunch & cafe & bistro & pastry

In [ ]:
fast_food & burger

In [ ]:
pizzy & pasta & italian 

In [15]:
dadata['type'].unique()


array(['brunch', 'cafe', 'mediterranean', 'european', 'chicken', 'bar',
       'seafood', 'south_america', 'veggie_healthy', 'bistro', 'grill',
       'indian_nepalese', 'portuguese', 'pizza', 'japanese', 'pasta',
       'mexican', 'africa_me', 'burger', 'italian', 'asian', 'pastry',
       'fast_food', 'fado'], dtype=object)

In [13]:
data['type'] = data['type'].map(lambda x : dict_gather_cats[x])

KeyError: 'south_america'

In [ ]:
def neighbours_df(neighbors, X):
    
    for n in neighbors: 
        for n in X.index:
            X.LOC[n]
        
         = X..loc[place].values[0]